# Graph Attention Networks

**WARNING**<br>
Execute only the colab

In [1]:
# install torch-geometric and related libraries
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 92.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.2/616.2 KB 24.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.3.0-py3-none-any.whl size=909897 sha256=a5e795025c05e6fea884e051b529e4df92e5a140c4e9670068cc9c2d58cf95cc
  Stored in directory: /root/.cache/pip/wheels/cd/7d/6b/17150450b80b4a3656a84330e22709ccd8dc0f8f4773ba4133
Succes

## Load the data

In [2]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

# load dataset
dataset = TUDataset(root='data/TUDataset', name='MUTAG')

# split dataset into train, validation and test
dataset = dataset.shuffle()
train_dataset = dataset[:140]
val_dataset = dataset[140:]

# create dataloader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

Extracting data/TUDataset/MUTAG/MUTAG.zip
Processing...
Done!
/usr/local/lib/python3.9/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


## Build the model

In [5]:
# define model
# import libraries
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv, global_mean_pool

# set parameters
n_features = dataset.num_features
n_hidden = 64
n_heads = 32
n_classes = dataset.num_classes

# define model class
class GAT(nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.gat1 = GATConv(n_features, n_hidden, heads=n_heads)
        self.gat2 = GATConv(n_hidden * n_heads, n_hidden, heads=n_heads)
        self.fc1 = nn.Linear(n_hidden * n_heads, n_classes)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = data.batch

        x = self.gat1(x, edge_index)
        x = self.relu(x)
        x = self.gat2(x, edge_index)
        x = self.relu(x)
        x = global_mean_pool(x, batch)
        x = self.dropout(x)
        x = self.fc1(x)

        return x
    
net = GAT()
net.cuda()

GAT(
  (gat1): GATConv(7, 64, heads=32)
  (gat2): GATConv(2048, 64, heads=32)
  (fc1): Linear(in_features=2048, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

## Train the model

In [6]:
# define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters())

# define function to evaluate model
def evaluate(loader):
    correct = 0
    total = 0
    for data in loader:
        data = data.cuda()
        outputs = net(data)
        _, predicted = torch.max(outputs.data, 1)
        total += data.y.size(0)
        correct += (predicted == data.y).sum().item()
    return correct / total * 100

# train model
n_epochs = 200
for epoch in range(n_epochs):
    for i, data in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        outputs = net(data)
        loss = criterion(outputs, data.y)
        loss.backward()
        optimizer.step()
    net.eval()
    train_acc = evaluate(train_loader)
    val_acc = evaluate(val_loader)
    print(f'Epoch {epoch+1}/{n_epochs}, Train Acc: {train_acc:.2f}%, Val Acc: {val_acc:.2f}%')

Epoch 1/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 2/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 3/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 4/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 5/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 6/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 7/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 8/200, Train Acc: 67.14%, Val Acc: 64.58%
Epoch 9/200, Train Acc: 67.14%, Val Acc: 66.67%
Epoch 10/200, Train Acc: 67.86%, Val Acc: 66.67%
Epoch 11/200, Train Acc: 72.14%, Val Acc: 68.75%
Epoch 12/200, Train Acc: 75.71%, Val Acc: 68.75%
Epoch 13/200, Train Acc: 72.14%, Val Acc: 79.17%
Epoch 14/200, Train Acc: 72.14%, Val Acc: 79.17%
Epoch 15/200, Train Acc: 76.43%, Val Acc: 77.08%
Epoch 16/200, Train Acc: 75.00%, Val Acc: 75.00%
Epoch 17/200, Train Acc: 72.14%, Val Acc: 77.08%
Epoch 18/200, Train Acc: 75.71%, Val Acc: 77.08%
Epoch 19/200, Train Acc: 75.71%, Val Acc: 75.00%
Epoch 20/200, Train Acc: 75.00%, Val Acc: 75.00%
Epoch 21/200, Train Acc: 76.4

## Evaluate the model

In [7]:
# evaluate model
net.eval()
test_acc = evaluate(val_loader)
print(f'Test Acc: {test_acc:.2f}%')

Test Acc: 75.00%
